# `montecarlo.py`

This notebook tests the `montecarlo` module.

This module contains methods to perform Monte Carlo integrals using FIGARO reconstructions.

## `MC_integral`

This method computes an integral of the form\
$
I = \int p(x)q(x)dx\,,
$\
where $p(x)$ and $q(x)$ are probability densities via the Monte Carlo approximation:\
$
I \simeq \frac{\sum_{i}p(x_i)}{N},\ x_i\sim q(x)\,.
$\
`p` and `q` can be either single FIGARO draws or lists of draws.

Let's compute the integral of two 1D Gaussian distributions. First of all, we need the FIGARO reconstructions:

In [ ]:
from figaro.mixture import DPGMM
from scipy.stats import norm
import numpy as np
from tqdm import tqdm

xmin = -5
xmax = 5

n_samps = 1000
n_draws = 100

mu1 = 1
mu2 = 0
s1  = 0.5
s2  = 1

f1  = norm(mu1,s1)
f2  = norm(mu2,s2)

x   = np.linspace(xmin, xmax, 1000)
dx  = x[1]-x[0]
mix = DPGMM([[xmin, xmax]])

samples_1 = f1.rvs(n_samps)
samples_2 = f2.rvs(n_samps)

draws_1 = np.array([mix.density_from_samples(samples_1) for _ in tqdm(range(n_draws), desc = 'Mixture 1')])
draws_2 = np.array([mix.density_from_samples(samples_2) for _ in tqdm(range(n_draws), desc = 'Mixture 2')])

I_num = np.sum(f1.pdf(x)*f2.pdf(x)*dx)

The integral can be done using the `MC_integral` method:

In [ ]:
from figaro.montecarlo import MC_integral

I, dI = MC_integral(draws_1, draws_2)

print('FIGARO: {0:.4f}+-{1:.4f}, numerical: {2:.4f}'.format(I, dI, I_num))

If one is not interested in the error estimate, just set the `error` argument to `False`:

In [ ]:
I = MC_integral(draws_1, draws_2, error = False)

print('FIGARO: {0:.4f}, numerical: {1:.4f}'.format(I, I_num))

This method works with every class with `pdf` and `rvs`, like `scipy.stats.norm`: 

In [ ]:
from figaro.montecarlo import MC_integral

I, dI = MC_integral(f1, f2)

print('FIGARO: {0:.4f}+-{1:.4f}, numerical: {2:.4f}'.format(I, dI, I_num))